In [6]:
import pandas as pd
import datetime
from dateutil.relativedelta import relativedelta
import requests
import json
from main import GoogleAPIClient

# Function

In [7]:
class GoogleSheets(GoogleAPIClient):
    def __init__(self) -> None:
        # 呼叫 GoogleAPIClient.__init__()，並提供 serviceName, version, scope
        super().__init__(
            'sheets',
            'v4',
            ['https://www.googleapis.com/auth/spreadsheets'],
        )

    def getWorksheet2Df(self, spreadsheetId: str, range: str):#獲取指定google sheet的檔案，輸出成dataframe
        request = self.googleAPIService.spreadsheets().values().get(
            spreadsheetId=spreadsheetId,
            range=range,
        )
        result = request.execute()['values']
        header = result[0]
        del result[0]
        return pd.DataFrame(result, columns=header)
    
    def getWorksheet(self, spreadsheetId: str, range: str):#獲取指定google sheet的檔案，輸出成list
        request = self.googleAPIService.spreadsheets().values().get(
            spreadsheetId=spreadsheetId,
            range=range,
        )
        response = request.execute()['values']
        return response

    def clearWorksheet(self, spreadsheetId: str, range: str):#清除指定google sheet的資料
        self.googleAPIService.spreadsheets().values().clear(
            spreadsheetId=spreadsheetId,
            range=range,
        ).execute()
        return 'done'
    
    def setWorksheet(self, spreadsheetId: str, range: str, df: pd.DataFrame):#清除指定google sheet的資料後新增資料(dataframe)
        self.clearWorksheet(spreadsheetId, range)
        #使用 update 的語法把我們新的數據加入至 Google Sheets
        self.googleAPIService.spreadsheets().values().update(
            spreadsheetId=spreadsheetId,
            range=range,
            valueInputOption='USER_ENTERED',
            body={
                'majorDimension': 'ROWS',
                'values': df.T.reset_index().T.values.tolist()
            },
        ).execute()
        return 'done'

    def appendWorksheet(self, spreadsheetId: str, range: str, df: pd.DataFrame):#對指定google sheet加入資料(dataframe)
        self.googleAPIService.spreadsheets().values().append(
            spreadsheetId=spreadsheetId,
            range=range,
            valueInputOption='USER_ENTERED',
            body={
                'majorDimension': 'ROWS',
                'values': df.values.tolist()
            },
        ).execute()
        return 'done'
    

In [ ]:
#分組規則測試
def testFun(pageName, url, df):
    d = df[(df["頁面分組"].str.contains(pageName))]
    l = len(d[(d['URL'].str.contains(url))])
    print(f'{pageName}總共有{len(d)}個，\n有{l}個\n包含{url}')
    ex = '\n'.join(d[~(d['URL'].str.contains(url))].URL)
    print(f'沒有包含到的URL是\n{ex}\n')
    
    e = df[(df["URL"].str.contains(url)==True)]
    l2 = e[~(e['頁面分組'].str.contains(pageName))]
    other = '\n'.join(l2.頁面分組)
    print(f'網址中有上面的URL的數據，不是"{pageName}"的有{len(l2)}個\n分類為\n{other}')

In [ ]:
#其他頁面測試
def otherPageTest(cat):
    sheetName = '網址分類_'+cat
    role = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshape.xlsx" ,sheet_name=sheetName)
    role['URL'] = role['URL'].str.lower()
    regex = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name=cat)
    claList = []
    for i in regex.reg:
        claList.extend(role[(role["URL"].str.contains(i)==True)].index)
    fileName = cat+'其他頁面.xlsx'
    role.iloc[list(set(role.index)-set(claList))].reset_index(drop = True).to_excel(fileName)

* 3/1抓2月會沒資料，之後要過幾天再抓前一個月
* 函式要改回dat - relativedelta(months=1)

In [3]:
def trafficGet(curBrand):
    dat = datetime.datetime.now()
    lastMonth = dat - relativedelta(months=2)
    lastMonth4api = lastMonth.strftime('%Y-%m')
    brand = curBrand

    url = "https://api.similarweb.com/v1/website/%s.com.tw/traffic-sources/overview-share?api_key=7e552f683942499fbd752ad182af1dab&start_date=%s&end_date=%s&country=tw&granularity=monthly&main_domain_only=false&format=json&mtd=false"%(brand, lastMonth4api, lastMonth4api)
    headers = {"accept": "application/json"}
    response = json.loads(requests.get(url, headers=headers).text)

    curUrl = brand + '.com.tw'
    trafficData = response['visits'][curUrl][0]['visits'][0]['organic']
    return trafficData

In [29]:
def classti(brand, oriDf):
    oriDf['URL'] = oriDf['URL'].str.lower()
    
    if brand=='518':
        #辨識其他頁面
        other518 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='518other')
        for i in range(len(other518.reg)):
            oriDf.loc[(oriDf['URL'].str.contains(other518.reg[i])), 'category'] = '其他頁面'
        #網頁群組分類
        regex518 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='518')
        catNum = len(regex518.cat)
        for i in range(catNum):
            if regex518.cat[i]=='03.職缺頁':
                searchResReg = regex518.loc[(regex518['cat'].str.contains('04.職缺搜尋結果'))].reg.values[0]
                oriDf.loc[~(oriDf['URL'].str.contains(searchResReg)) & (oriDf['URL'].str.contains('^518.com.tw/job-')), 'category'] = '03.職缺頁'
                oriDf.loc[~(oriDf['URL'].str.contains(searchResReg)) & (oriDf['URL'].str.contains('^518.com.tw/job-')), 'index'] = 3
            else:
                oriDf.loc[(oriDf['URL'].str.contains(regex518.reg[i])), 'category'] = regex518.cat[i]
                oriDf.loc[(oriDf['URL'].str.contains(regex518.reg[i])), 'index'] = regex518.indexC[i]
        #辨識未分類頁面
        oriDf.loc[oriDf.category.isnull(), 'category'] = '未分類'
        
        oriDf.loc[oriDf['category']=='未分類', 'index'] = 0
        oriDf.loc[oriDf['category']=='其他頁面', 'index'] = len(regex518)+1
    
    elif brand=='1111':
        #辨識其他頁面
        other1111 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='1111other')
        for i in range(len(other1111.reg)):
            oriDf.loc[(oriDf['URL'].str.contains(other1111.reg[i])), 'category'] = '其他頁面'
        #網頁群組分類
        regex1111 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='1111')
        catNum = len(regex1111.cat)
        for i in range(catNum):
            oriDf.loc[(oriDf['URL'].str.contains(regex1111.reg[i])), 'category'] = regex1111.cat[i]
            oriDf.loc[(oriDf['URL'].str.contains(regex1111.reg[i])), 'index'] = regex1111.indexC[i]
        #辨識未分類頁面   
        oriDf.loc[oriDf.category.isnull(), 'category'] = '未分類'
        
        oriDf.loc[oriDf['category']=='未分類', 'index'] = 0
        oriDf.loc[oriDf['category']=='其他頁面', 'index'] = len(regex1111)+1
        
    elif brand=='104':
        #辨識其他頁面
        other104 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='104other')
        for i in range(len(other104.reg)):
            oriDf.loc[(oriDf['URL'].str.contains(other104.reg[i])), 'category'] = '其他頁面'
        #網頁群組分類
        regex104 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='104')
        catNum = len(regex104.cat)
        for i in range(catNum):
            if regex104.cat[i]=='招募管理':
                searchResReg = regex104.loc[(regex104['cat'].str.contains('人資充電'))].reg.values[0]
                oriDf.loc[~(oriDf['URL'].str.contains(searchResReg)) & (oriDf['URL'].str.contains('^pro.104.com.tw')), 'category'] = '招募管理'
                oriDf.loc[~(oriDf['URL'].str.contains(searchResReg)) & (oriDf['URL'].str.contains('^pro.104.com.tw')), 'index'] = 25
            else:
                oriDf.loc[(oriDf['URL'].str.contains(regex104.reg[i])), 'category'] = regex104.cat[i]
                oriDf.loc[(oriDf['URL'].str.contains(regex104.reg[i])), 'index'] = regex104.indexC[i]
        #辨識未分類頁面
        oriDf.loc[oriDf.category.isnull(), 'category'] = '未分類'
        
        oriDf.loc[oriDf['category']=='未分類', 'index'] = 0
        oriDf.loc[oriDf['category']=='其他頁面', 'index'] = len(regex104)+1
        
    else:
        oriDf['category'] = ['目前暫無此網頁分類規則']*len(oriDf)
        oriDf['index'] = [0]*len(oriDf)

#     oriDf = oriDf.sort_values(by=['index'])
    oriDf['checkIndex'] = [i for i in range(len(oriDf))]
    #日期欄位設定
    dat = datetime.datetime.now()
    lastMonth = dat - relativedelta(months=1)
    lastMonthStr = lastMonth.strftime('%b-%y')
    oriDf['date'] = lastMonthStr
    
    #計算各網頁traffic
    tra = 847054.8980801627#trafficGet(brand)
    oriDf['traffic'] = oriDf['Traffic Share']*tra
    
    #欄位順序調整
    oriDf = oriDf[['checkIndex', 'date', 'URL', 'Traffic Share', 'Keywords', 'Top Keyword', 'category', 'traffic']]
    
    return oriDf

In [5]:
# 分類成果測試
def resTestFun(pageName, url, df):
    d = df[(df["category"].str.contains(pageName))]
    l = len(d[(d['URL'].str.contains(url))])
    print(f'{pageName}總共有{len(d)}個，\n有{l}個\n包含{url}')
    ex = '\n'.join(d[~(d['URL'].str.contains(url))].URL)
    print(f'沒有包含到的URL是\n{ex}\n')
    
    e = df[(df["URL"].str.contains(url)==True)]
    l2 = e[~(e['category'].str.contains(pageName))]
    other = '\n'.join(l2.category)
    print(f'網址中有上面的URL的數據，不是"{pageName}"的有{len(l2)}個\n分類為\n{other}')

# 網頁URL紀錄

In [235]:
role1111 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshape.xlsx" ,sheet_name='網址分類_1111')
role104 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshape.xlsx" ,sheet_name='網址分類_104')
role518 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshape.xlsx" ,sheet_name='網址分類_518')

In [236]:
role1111['URL'] = role1111['URL'].str.lower()
role104['URL'] = role104['URL'].str.lower()
role518['URL'] = role518['URL'].str.lower()

In [455]:
testFun('01.首頁', '^1111.com.tw$', role1111)

01.首頁總共有1個，
有1個
包含^1111.com.tw$
沒有包含到的URL是


網址中有上面的URL的數據，不是"01.首頁"的有0個
分類為



In [350]:
e = role104[(role104["URL"].str.contains('^104.com.tw/jb/category|^104.com.tw/jobs/main/category|^104.com.tw/jobs/main/freshman')==True)]
l2 = e[~(e['頁面分組'].str.contains('06.分類找工作'))]
l2

,URL,頁面分組
35345,104.com.tw/jobs/main/freshman,其他頁面
38800,104.com.tw/jobs/main/category,其他頁面
40354,104.com.tw/jobs/main/category,03.職缺頁
40355,104.com.tw/jobs/main/category?area=6002000000,03.職缺頁
40356,104.com.tw/jobs/main/freshman,03.職缺頁


In [522]:
#測試用正規表達後，有沒有甚麼URL分類不應該在其他頁面的被包含進來
regex518 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='518')
regex1111 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='1111')
regex104 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='104')

In [532]:
otherPageTest('518')

# 頁面分組判斷

## 518

In [8]:
oriDf518 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/ori518.xlsx", sheet_name='Landing_Pages')
regex518 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='518')

In [9]:
catNum = len(regex518.cat)
oriDf518['URL'] = oriDf518['URL'].str.lower()
for i in range(catNum):
    if regex518.cat[i]=='03.職缺頁':
        searchResReg = regex518.loc[(regex518['cat'].str.contains('04.職缺搜尋結果'))].reg.values[0]
        oriDf518.loc[~(oriDf518['URL'].str.contains(searchResReg)) & (oriDf518['URL'].str.contains('^518.com.tw/job-')), 'category'] = '03.職缺頁'
    else:
        oriDf518.loc[(oriDf518['URL'].str.contains(regex518.reg[i])), 'category'] = regex518.cat[i]

In [10]:
other518 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='518other')
oriDf518.loc[oriDf518.URL.isin(other518.reg.values), 'category'] = '其他頁面'
oriDf518.loc[oriDf518.category.isnull(), 'category'] = '未分類'

In [408]:
# oriDf518.to_excel('518test.xlsx')

## 1111

In [11]:
oriDf1111 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/ori1111.xlsx", sheet_name='Landing_Pages')
regex1111 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='1111')

In [12]:
catNum = len(regex1111.cat)
oriDf1111['URL'] = oriDf1111['URL'].str.lower()
for i in range(catNum):
    oriDf1111.loc[(oriDf1111['URL'].str.contains(regex1111.reg[i])), 'category'] = regex1111.cat[i]

other1111 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='1111other')
oriDf1111.loc[oriDf1111.URL.isin(other1111.reg.values), 'category'] = '其他頁面'
oriDf1111.loc[oriDf1111.category.isnull(), 'category'] = '未分類'

In [494]:
# oriDf1111.to_excel('1111test.xlsx')

## 104

In [13]:
oriDf104 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/ori104.xlsx", sheet_name='Landing_Pages')
regex104 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='104')

In [14]:
catNum = len(regex104.cat)
oriDf104['URL'] = oriDf104['URL'].str.lower()
for i in range(catNum):
    if regex104.cat[i]=='招募管理':
        searchResReg = regex104.loc[(regex104['cat'].str.contains('人資充電'))].reg.values[0]
        oriDf104.loc[~(oriDf104['URL'].str.contains(searchResReg)) & (oriDf104['URL'].str.contains('^pro.104.com.tw')), 'category'] = '招募管理'
    else:
        oriDf104.loc[(oriDf104['URL'].str.contains(regex104.reg[i])), 'category'] = regex104.cat[i]

other104 = pd.read_excel("C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/cat.xlsx" ,sheet_name='104other')
oriDf104.loc[oriDf104.URL.isin(other104.reg.values), 'category'] = '其他頁面'
oriDf104.loc[oriDf104.category.isnull(), 'category'] = '未分類'

In [552]:
# oriDf104.to_excel('104test.xlsx')

# 分類成果測試

## 測試1:雙向測試

In [551]:
for i in range(len(regex104)):
    print(resTestFun(regex104.cat[i], regex104.reg[i], oriDf104),'\n')

01.首頁總共有2個，
有2個
包含^104.com.tw$|^104.com.tw/jobs/main$|^104.com.tw/104i/introduce.cfm$
沒有包含到的URL是


網址中有上面的URL的數據，不是"01.首頁"的有0個
分類為

None 

02.公司頁總共有2646個，
有2646個
包含^104.com.tw/company/|^104.com.tw/company$
沒有包含到的URL是


網址中有上面的URL的數據，不是"02.公司頁"的有0個
分類為

None 

03.職缺頁總共有224個，
有224個
包含^104.com.tw/job/|^m.104.com.tw/job/|^104.com.tw/jobs/apply/analysis|^104.com.tw/jobs/career_whitepaper
沒有包含到的URL是


網址中有上面的URL的數據，不是"03.職缺頁"的有0個
分類為

None 

04.職缺搜尋結果總共有103個，
有103個
包含^104.com.tw/jobs/search|^m.104.com.tw/search|^m.104.com.tw/custsearch/
沒有包含到的URL是


網址中有上面的URL的數據，不是"04.職缺搜尋結果"的有0個
分類為

None 

05.公司搜尋結果總共有0個，
有0個
包含^104.com.tw/company\?|^m.104.com.tw/custsearch/custlist
沒有包含到的URL是


網址中有上面的URL的數據，不是"05.公司搜尋結果"的有0個
分類為

None 

06.分類找工作總共有1個，
有1個
包含^104.com.tw/jb/category|^104.com.tw/jobs/main/category|^104.com.tw/jobs/main/freshman
沒有包含到的URL是


網址中有上面的URL的數據，不是"06.分類找工作"的有0個
分類為

None 

104i_公司頁總共有0個，
有0個
包含^104.com.tw/jb/104i/cust/view
沒有包含到的URL是


網址中有上面的URL的數據，不是"104i_公司頁"的有0個
分類為

None 

1

In [546]:
e = oriDf104[(oriDf104["URL"].str.contains('^104.com.tw/job/|^m.104.com.tw/job/|^104.com.tw/jobs/apply/analysis|^104.com.tw/jobs/career_whitepaper')==True)]
l2 = e[~(e['category'].str.contains('03.職缺頁'))]
l2

,URL,Traffic Share,Keywords,Top Keyword,category
9,104.com.tw/job/77icd,0.002444,6,pcie ssd 韌體開發工程師0503,未分類
195,104.com.tw/job/4of03,0.000417,10,群聯 台北 12f,未分類
231,104.com.tw/job/6mj4r,0.000390,11,pcie gen4/5 ssd韌體工程師 d2 – 台北辦公室,未分類
338,104.com.tw/job/7avdb,0.000329,2,麻豆燦坤門市,未分類
351,104.com.tw/job/7ewhe,0.000323,1,軟韌體工程師1601,未分類
...,...,...,...,...,...
3682,104.com.tw/job/26d60,0.000040,1,synology product specialist,未分類
3702,104.com.tw/job/6diey,0.000040,2,英業達 品質人員,未分類
3710,104.com.tw/job/7s7ft,0.000040,2,esg analyst taiwan,未分類
3748,104.com.tw/job/7u09c,0.000038,1,頂通 翻譯,未分類


## 測試2:過去資料測試

In [15]:
test1111 = pd.read_excel('C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshapeTest.xlsx', sheet_name='1111')[['URL','Traffic Share',"Keywords",'Top Keyword']].reset_index()
test518 = pd.read_excel('C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshapeTest.xlsx', sheet_name='518')[['URL','Traffic Share',"Keywords",'Top Keyword']].reset_index()
test104 = pd.read_excel('C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshapeTest.xlsx', sheet_name='104')[['URL','Traffic Share',"Keywords",'Top Keyword']].reset_index()

In [46]:
result1111 = classti('1111', test1111)
result518 = classti('518', test518)
result104 = classti('104', test104)

In [33]:
ans1111 = pd.read_excel('C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshapeTest.xlsx', sheet_name='1111')[['URL', '頁面分組']].reset_index()
ans518 = pd.read_excel('C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshapeTest.xlsx', sheet_name='518')[['URL', '頁面分組']].reset_index()
ans104 = pd.read_excel('C:/Users/wendywen/python code/06-202208-AutoUpdateKPIReport/sw檔案/reshapeTest.xlsx', sheet_name='104')[['URL', '頁面分組']].reset_index()
ans1111.columns = ['index', 'URL', 'catAns']
ans518.columns = ['index', 'URL', 'catAns']
ans104.columns = ['index', 'URL', 'catAns']

In [47]:
merDf1111 = pd.merge(ans1111, result1111, left_on='index', right_on='checkIndex')
merDf104 = pd.merge(ans104, result104, left_on='index',right_on = 'checkIndex')
merDf518 = pd.merge(ans518, result518, left_on='index',right_on = 'checkIndex')

In [48]:
errorIndex1111 = []
for i in range(len(merDf1111)):
    if merDf1111.catAns[i] != merDf1111.category[i]:
        errorIndex1111.append(i)
    else:
        continue
        
errorIndex104 = []
for i in range(len(merDf104)):
    if merDf104.catAns[i] != merDf104.category[i]:
        errorIndex104.append(i)
    else:
        continue
        
errorIndex518 = []
for i in range(len(merDf518)):
    if merDf518.catAns[i] != merDf518.category[i]:
        errorIndex518.append(i)
    else:
        continue

In [49]:
file = pd.ExcelWriter('error.xlsx')
merDf1111.iloc[errorIndex1111].to_excel(file, sheet_name = 'error1111')
merDf104.iloc[errorIndex104].to_excel(file, sheet_name = 'error104')
merDf518.iloc[errorIndex518].to_excel(file, sheet_name = 'error518')
file.save()
file.close()

C:\ProgramData\Anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


# 以API抓取Traffic數據

dat = datetime.datetime.now()
lastMonth = dat - relativedelta(months=1)
lastMonth4api = lastMonth.strftime('%Y-%m')
brand = '1111'

url = "https://api.similarweb.com/v1/website/%s.com.tw/traffic-sources/overview-share?api_key=7e552f683942499fbd752ad182af1dab&start_date=%s&end_date=%s&country=tw&granularity=monthly&main_domain_only=false&format=json&mtd=false"%(brand, lastMonth4api, lastMonth4api)
headers = {"accept": "application/json"}
response = json.loads(requests.get(url, headers=headers).text)

curUrl = brand+'.com.tw'
response['visits'][curUrl][0]['visits'][0]['organic']

In [775]:
response['visits']['1111.com.tw'][0]['visits'][0]['organic']

847054.8980801627

In [27]:
result1111 = classti('1111', test1111)

In [28]:
result1111#[['date', 'URL', 'Traffic Share', 'Keywords', 'Top Keyword', 'category', 'traffic']]

,index,date,URL,Traffic Share,Keywords,Top Keyword,category,traffic
0,1,Feb-23,1111.com.tw,0.166265,9179,1111,01.首頁,140835.272708
1,2,Feb-23,1111.com.tw/corp/10000670,0.000267,13,國衛院 行政人員 薪水,02.公司頁,226.158832
2,2,Feb-23,1111.com.tw/corp/10001567,0.000469,1,雙永股份有限公司 職缺,02.公司頁,397.491281
3,2,Feb-23,1111.com.tw/corp/10067687,0.000319,2,中鋼運通,02.公司頁,270.215747
4,2,Feb-23,1111.com.tw/corp/10342078,0.000403,6,榮興工業,02.公司頁,341.685855
...,...,...,...,...,...,...,...,...
26186,21,Feb-23,1111.com.tw/position/nurse,0.000884,186,護理師,職點,749.112956
26187,21,Feb-23,1111.com.tw/position/product-manager,0.000679,338,產品經理,職點,574.771046
26188,21,Feb-23,1111.com.tw/position/reporter,0.000267,43,記者 能力,職點,226.087226
26189,21,Feb-23,1111.com.tw/position/sales,0.000475,750,業務,職點,402.021300


In [818]:
#原始數據
orgDf = GoogleSheets().getWorksheet2Df(
        spreadsheetId='1YWi8hg545ryn_KlWUSp2jDKt85BZ1ACHGw67iwIyrgY',
        range='1111'
)

#原始數據
GoogleSheets().setWorksheet(
        spreadsheetId='1YWi8hg545ryn_KlWUSp2jDKt85BZ1ACHGw67iwIyrgY',
        range='1111',
        df = pd.concat([result1111, orgDf], axis=0)#.fillna('')
)

'done'

# 使用者介面

In [738]:
import tkinter as tk
from tkinter import filedialog
import pandas as pd

class Application(tk.Frame):
    def __init__(self, master=None):
        super().__init__(master)
        self.master = master
        self.master.title("Excel匯入程式")
        self.master.geometry("400x200")
        self.create_widgets()

    def create_widgets(self):
        self.excel_label = tk.Label(self.master, text="請選擇Excel檔案：")
        self.excel_label.grid(row=0, column=0, padx=10, pady=10)
        self.excel_button = tk.Button(self.master, text="選擇檔案", command=self.choose_file)
        self.excel_button.grid(row=0, column=1, padx=10, pady=10)

        self.brand_label = tk.Label(self.master, text="輸入檔案主題：")
        self.brand_label.grid(row=1, column=0, padx=10, pady=10)
        self.brand_entry = tk.Entry(self.master)
        self.brand_entry.grid(row=1, column=1, padx=10, pady=10)

        self.traffic_label = tk.Label(self.master, text="輸入數字：")
        self.traffic_label.grid(row=2, column=0, padx=10, pady=10)
        self.traffic_entry = tk.Entry(self.master)
        self.traffic_entry.grid(row=2, column=1, padx=10, pady=10)

        self.submit_button = tk.Button(self.master, text="匯入Excel檔", command=self.load_excel)
        self.submit_button.grid(row=3, column=1, padx=10, pady=10)

    def choose_file(self):
        filetypes = (("Excel files", "*.xlsx"), ("All files", "*.*"))
        self.filename = filedialog.askopenfilename(filetypes=filetypes)
        self.excel_button.configure(text=self.filename)

    def load_excel(self):
        brand = self.brand_entry.get()
        traffic = int(self.traffic_entry.get())
        oriTest = pd.read_excel(self.filename, sheet_name = 'Landing_Pages')

        # 在這裡可以進行DataFrame的操作，如顯示前5行
        print(oriTest.head())

root = tk.Tk()
app = Application(master=root)
app.mainloop()
